<a href="https://colab.research.google.com/github/kienprojects/Machine-Learning-course/blob/main/01_Preprocessing_News.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pre-processing News Data

## Bài toán
Dữ liệu gồm n văn bản phân vào 10 chủ đề khác nhau. Cần biễu diễn mỗi văn bản dưới dạng một vector số thể hiện cho nội dụng của văn bản đó.

## Sử dụng phương pháp mã hóa: TF-IDF
Cho tập gồm $n$ văn bản: $D = \{d_1, d_2, ... d_n\}$. Tập từ điển tương ứng được xây dựng từ $n$ văn bản này có độ dài là $m$
- Xét văn bản $d$ có $|d|$ từ và $t$ là một từ trong $d$. Mã hóa tf-idf của $t$ trong văn bản $d$ được biểu diễn:
\begin{equation}
    \begin{split}
        \text{tf}_{t, d} &= \frac{f_t}{|d|} \\
        \text{idf}_{t, d} &= \log\frac{n}{n_t}, \ \ \ \ n_t = |\{d\in D: t\in d\}| \\
        \text{tf-idf}_{t d} &= \text{tf}_{t, d} \times \text{idf}_{t, d}
    \end{split}
\end{equation}

- Khi đó văn bản $d$ được mã hóa là một vector $m$ chiều. Các từ xuất hiện trong d sẽ được thay bằng giá trị tf-idf tương ứng. Các từ không xuất hiện trong $d$ thì thay là 0

# Các bước làm

## Chuẩn bị các thư viện cần thiết

In [1]:
!pip install pyvi

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 30.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 21.4 MB/s eta 0:00:00


In [2]:
import os
import matplotlib.pyplot as plt
import numpy as np

from sklearn.datasets import load_files
from pyvi import ViTokenizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline

%matplotlib inline

In [4]:
# from google.colab import drive
# drive.mount('/content/drive')
%cd /content/drive/MyDrive/VINBIGDATA/MachineLearning/00_Assignment/01_Preprocessing/02_News_VNExpress

/content/drive/.shortcut-targets-by-id/1TvCG3X3dcbtWR5iQlJl5HnUNGb5lsmIA/00_Assignment/01_Preprocessing/02_News_VNExpress


## Load dữ liệu từ thư mục

Cấu trúc thư mục như sau

- data/news_vnexpress/

    - Kinh tế:
        - bài báo 1.txt
        - bài báo 2.txt
    - Pháp luật
        - bài báo 3.txt
        - bài báo 4.txt

In [6]:
#Có thể chỉnh lại đường dẫn thư mục cho phù hợp
INPUT = 'news_vnexpress'
os.makedirs("/images",exist_ok=True)  # thư mục lưu các các hình ảnh trong quá trình huấn luyện và đánh gía

In [7]:
# statistics
print('Các nhãn và số văn bản tương ứng trong dữ liệu')
print('----------------------------------------------')
n = 0
for label in os.listdir(INPUT):
    print(f'{label}: {len(os.listdir(os.path.join(INPUT, label)))}')
    n += len(os.listdir(os.path.join(INPUT, label)))

print('-------------------------')
print(f"Tổng số văn bản: {n}")

Các nhãn và số văn bản tương ứng trong dữ liệu
----------------------------------------------
doi-song: 120
phap-luat: 59
thoi-su: 59
the-thao: 135
giao-duc: 105
du-lich: 54
-------------------------
Tổng số văn bản: 532


In [8]:
# load data
data_train = load_files(container_path=INPUT, encoding="utf-8")
print('mapping:')
for i in range(len(data_train.target_names)):
    print(f'{data_train.target_names[i]} - {i}')

print('--------------------------')
print(data_train.filenames[0:1])
# print(data_train.data[0:1])j
print(data_train.target[0:1])
print(data_train.data[0:1])

print("\nTổng số  văn bản: {}" .format( len(data_train.filenames)))

mapping:
doi-song - 0
du-lich - 1
giao-duc - 2
phap-luat - 3
the-thao - 4
thoi-su - 5
--------------------------
['news_vnexpress/the-thao/00076.txt']
[4]
['"Anh bảo ‘tưởng nhớ Tiger ư?’. Anh ấy bị tai nạn nghiêm trọng. Tôi xem vài bản tin và thấy họ nói cứ như anh ấy chết rồi. Nhưng Tiger chưa mất mà. Chúng ta rất may khi anh ấy còn sống. Chúng ta nên ca ngợi những công trạng vô giá mà anh ấy mang lại cho PGA Tour và cả golf", McIlroy đáp khi một phóng viên gợi ý tưởng nhớ Woods.Năm 2017, McIlroy có dịp ăn trưa cùng Woods, ngay sau lần golfer Mỹ lần thứ tư phẫu thuật đốt sống. Qua lần đó, golfer Bắc Ireland nói chỉ những người thân cận mới hiểu được nỗi khổ sở của Woods trong hành trình bình phục.Sau vụ tai nạn nghiêm trọng khiến Woods chịu đa chấn thương hôm 23/2, đã có một số ý kiến rằng golfer cựu số một thế giới vẫn có thể trở lại, như màn tái xuất ấn tượng khi thắng major Masters 2019. Tuy nhiên, McIlroy không tinThể thao thế giới đánh giá Woods thắng major Masters 2019 là cuộc t

## Chuyển dữ liệu dạng text về ma trận (n x m) bằng TF-IDF

* Bạn cần viết đoạn mã tương ứng trong cell bên dưới. Theo các bước được gợi ý

In [10]:
# load dữ liệu các stopwords

def load_stopwords(path):
    f = open(path, "r", encoding="utf-8")
    stopwords = f.readlines()
    stopwords = [x.strip() for x in stopwords]
    return stopwords

stopwords = load_stopwords("vietnamese-stopwords.txt")

In [15]:

# Chuyển hoá dữ liệu text về dạng vector TF
#     - loại bỏ từ dừng
#     - sinh từ điển
from sklearn.feature_extraction.text import TfidfVectorizer
def transform_data(data):
    removed_stopwords = []
    for text in data:
        removed_stopwords.append(ViTokenizer.tokenize(text))

    tfidf_vectorizer = TfidfVectorizer(stop_words=stopwords)
    tfidf_vectorizer.fit(removed_stopwords)
    return tfidf_vectorizer

tfidf_vectorizer = transform_data(data_train.data)


In [16]:


# Hàm thực hiện chuyển đổi dữ liệu text thành dữ liệu số dạng ma trận
# Input: Dữ liệu 2 chiều dạng numpy.array, mảng nhãn id dạng numpy.array

data_preprocessed = tfidf_vectorizer.transform(data_train.data)

X = data_preprocessed # thuoc tinh
Y = data_train.target #nhan

print(f"\nSố lượng từ trong từ điển: {len(module_count_vector.vocabulary_)}")
print(f"Kích thước dữ liệu sau khi xử lý: {X.shape}")
print(f"Kích thước nhãn tương ứng: {Y.shape}")


Số lượng từ trong từ điển: 14709
Kích thước dữ liệu sau khi xử lý: (532, 14709)
Kích thước nhãn tương ứng: (532,)


In [17]:
print(X[100].toarray())
print(Y[100])

[[0. 0. 0. ... 0. 0. 0.]]
2


In [18]:
print(X[100]) #Sau khi xử lí, dữ liệu được lưu dưới dạng ma trận thưa như sau:

  (0, 14525)	0.05060194794708251
  (0, 13935)	0.0950498607656479
  (0, 13880)	0.05164126818626985
  (0, 13558)	0.14625584604740072
  (0, 12957)	0.0820842008338501
  (0, 12943)	0.10804539899648732
  (0, 12819)	0.07703023151731735
  (0, 12643)	0.4925052050031006
  (0, 12370)	0.046769267386527535
  (0, 12270)	0.049642657658738765
  (0, 12253)	0.05402269949824366
  (0, 12246)	0.1540604630346347
  (0, 11696)	0.11750578838973479
  (0, 11150)	0.046769267386527535
  (0, 10755)	0.43646371381789817
  (0, 10727)	0.06083516021777726
  (0, 10330)	0.03236302243322569
  (0, 10261)	0.05875289419486739
  (0, 9899)	0.037947481256980546
  (0, 9179)	0.04918934872024151
  (0, 9021)	0.05219534260502073
  (0, 7573)	0.11393839253511057
  (0, 7543)	0.06480456560833703
  (0, 7439)	0.03888511277006741
  (0, 7280)	0.07703023151731735
  :	:
  (0, 4836)	0.052775197016465304
  (0, 4727)	0.12960913121667406
  (0, 4654)	0.10939325395054303
  (0, 4430)	0.019795838667142202
  (0, 4365)	0.06083516021777726
  (0, 4321)	0.